In [1]:
import pandas as pd
from google.colab import drive
# drive.mount('/content/drive')
df = pd.read_csv('/content/labeled_data.csv')

In [2]:
!pip install --upgrade transformers

In [3]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [4]:
# Step 1: Install dependencies (uncomment below if running in Colab)
# !pip install transformers datasets sklearn

# Step 2: Import libraries
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from transformers import TrainingArguments, IntervalStrategy

# Step 3: Load your dataset
df = pd.read_csv("labeled_data.csv")
df = df[['tweet', 'class']]  # 'class' contains labels 0, 1, 2
df = df.rename(columns={'tweet': 'text', 'class': 'label'})
df = df.dropna()

# Step 4: Train/validation split
train_df, val_df = train_test_split(df, test_size=0.1, stratify=df['label'], random_state=42)

# Step 5: Convert to Hugging Face Dataset
dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df.reset_index(drop=True)),
    'test': Dataset.from_pandas(val_df.reset_index(drop=True))
})

# Step 6: Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['text'])
tokenized_datasets.set_format("torch")

# Step 7: Load Model for 3-class classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# Step 8: Define metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')  # macro for imbalanced
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Step 9: Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    # evaluation_strategy="epoch",
    logging_dir='./logs',
    save_strategy="epoch",
    # load_best_model_at_end=True,
    # metric_for_best_model="f1",
    # greater_is_better=True,
    eval_steps=500
)

# Step 10: Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

# Step 11: Train the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gamingbuddy547 (gamingbuddy547-asdwwd) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.348900
1000,0.275200
1500,0.254000
2000,0.215300
2500,0.209700
3000,0.172200
3500,0.164800
4000,0.157600


TrainOutput(global_step=4182, training_loss=0.22172817939893302, metrics={'train_runtime': 1501.4809, 'train_samples_per_second': 44.564, 'train_steps_per_second': 2.785, 'total_flos': 4401361251753984.0, 'train_loss': 0.22172817939893302, 'epoch': 3.0})

In [5]:
# trainer.predict('You are a waste person, i swear your family')
# Prepare the input text for prediction
input_text = "fuck off man!!! such a dog you are..."
input_data = tokenizer(input_text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

# Convert input_data to a Dataset
import pandas as pd
from datasets import Dataset

df = pd.DataFrame([input_text], columns=['text'])  # Create a DataFrame with the input text
input_dataset = Dataset.from_pandas(df)  # Convert to Hugging Face Dataset
input_dataset = input_dataset.map(lambda examples: tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128), batched=True)  # Tokenize
input_dataset.set_format('pt')  # Set format to PyTorch tensors

# Make prediction
predictions = trainer.predict(input_dataset)

# Access prediction results (e.g., predicted class probabilities)
print(predictions.predictions)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

[[-0.6935862  3.7898622 -2.792014 ]]


In [6]:
import torch

# Get logits (raw prediction scores)
logits = predictions.predictions

# Apply argmax to get the predicted class
predicted_class = torch.argmax(torch.tensor(logits), dim=1).item()

# Print predicted class
print(f"Predicted class: {predicted_class}")


class_names = {0: "Hate Speech", 1: "Offensive Language", 2: "Neither"}
print(f"Predicted class: {predicted_class} -> {class_names[predicted_class]}")

Predicted class: 1
Predicted class: 1 -> Offensive Language


In [7]:
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Get true labels from the validation dataset
true_labels = tokenized_datasets['test']['label']

# Step 2: Get model predictions on the validation dataset
predictions_output = trainer.predict(tokenized_datasets['test'])

# Step 3: Convert logits to predicted class labels
predicted_labels = torch.argmax(torch.tensor(predictions_output.predictions), dim=1)

# Step 4: Compute accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Validation Accuracy: {accuracy:.4f}")

# Optional: Print classification report
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels, target_names=["Hate Speech", "Offensive Language", "Neither"]))

Validation Accuracy: 0.9242

Classification Report:
                    precision    recall  f1-score   support

       Hate Speech       0.59      0.39      0.47       143
Offensive Language       0.94      0.97      0.96      1920
           Neither       0.92      0.89      0.91       416

          accuracy                           0.92      2479
         macro avg       0.82      0.75      0.78      2479
      weighted avg       0.92      0.92      0.92      2479

